<a href="https://colab.research.google.com/github/jasonbuchanan145/Data-Modeling/blob/main/hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets
!pip install huggingface_hub
!pip install py-solc-x
!pip install -U bitsandbytes
!pip install slither-analyzer
!solc-select install 0.8.28
!pip install trl
!solc-select use 0.8.28

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have n

In [2]:
from google.colab import files

# Create a working directory for our Solidity project
!mkdir -p solidity_project
%cd solidity_project

# Initialize npm project and install dependencies
!npm init -y
!npm install --save-dev hardhat @nomicfoundation/hardhat-toolbox @openzeppelin/contracts
!npm install @chainlink/contracts # Add other common dependencies as needed


# Create contracts directory
!mkdir -p contracts

/content/solidity_project
Wrote to /content/solidity_project/package.json:

{
  "name": "solidity_project",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "description": ""
}



⠙⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧npm warn deprecated glob@8.1.0: Glob versions prior to v9 are no longer supported
⠧⠇⠏⠋⠙⠹⠸⠼npm warn deprecated ethereumjs-abi@0.6.8: This library has been deprecated and usage is discouraged.
⠼⠴⠦⠧⠇⠏⠋npm warn deprecated lodash.isequal@4.5.0: This package is deprecated. Use require('node:util').isDeepStrictEqual instead.
⠋⠙⠹⠸npm warn deprecated glob@7.2.3: Glob versions prior to v9 are no longer supported
⠸⠼npm warn deprecated glob@7.2.3: Glob versions prior to v9 are no longer supported
⠼npm warn deprecated glob@5.0.15:

In [3]:
import logging
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

# Create a file handler and set the filename
file_handler = logging.FileHandler('/content/logger.log')

# Create a formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the handler to the logger
log.addHandler(file_handler)
log.propagate = False

In [4]:
# Standard libraries
import os
import re
import json
import tempfile
import glob
import subprocess
import requests
import matplotlib.pyplot as plt
import hashlib
from pathlib import Path

# PyTorch and related
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.amp import autocast
from torch.amp import GradScaler

# Transformers and TRL
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TFAutoModelForMaskedLM,
    GenerationConfig,
    pipeline,
    BitsAndBytesConfig
)
from trl import (
    PPOTrainer,
    PPOConfig,
    AutoModelForCausalLMWithValueHead,
    create_reference_model
)

# Datasets
from datasets import load_dataset

# Solidity Compiler
from solcx import compile_standard, install_solc

# BitsAndBytes
from bitsandbytes.optim import Adam8bit

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# Set the path to save the best model
output_dir = "/content/drive/MyDrive/llama_generated_contracts/explanations"



In [5]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
model_name = "Qwen/Qwen2.5-Coder-7B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config
)
model.gradient_checkpointing_enable()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [6]:
# Create a new model instance on CPU
with torch.device("cpu"):
    reference_model = AutoModelForCausalLMWithValueHead.from_pretrained(
        model_name,
        device_map="cpu"
      )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
from torch.utils.data import Dataset
import torch.nn as nn
from trl.core import LengthSampler
from datasets import Dataset as HFDataset
class SimpleRewardModel(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleRewardModel, self).__init__()
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, hidden_states):
        # For this simple example, we average the hidden states over the sequence length.
        mean_hidden = hidden_states.mean(dim=1)  # shape: [batch, hidden_size]
        reward = self.linear(mean_hidden)         # shape: [batch, 1]
        return reward

In [8]:
def make_prompts(file_path):
  dataset = []
  with open(file_path, 'r') as file:
      for line in file:
          line = line.strip()
          if line.startswith('"') and line.endswith('"'):
              # Remove the surrounding quotes
              clean_line = line[1:-1]
              clean_line = "Using Solidity and having your code start with \"// Begin Smart Contract\" and ending with \"//End Smart Contract\" as comments "+clean_line
              dataset.append(clean_line)
  return dataset
prompts = make_prompts("/content/drive/MyDrive/prompts.txt")
validation_prompts = make_prompts("/content/drive/MyDrive/validation_prompts.txt")

In [9]:
def setup_hardhat_environment():
    """
    Set up the initial Hardhat environment with common dependencies.
    Run this once at the beginning of your notebook.
    """
    import subprocess
    import os

    # Create directories relative to the project root
    os.makedirs('./contracts', exist_ok=True)
    os.makedirs('./artifacts', exist_ok=True)

    log.info("Installing common Solidity dependencies...")
    dependencies = [
        "hardhat",
        "@nomicfoundation/hardhat-toolbox",
        "@openzeppelin/contracts",
        "@chainlink/contracts"
    ]

    install_cmd = ["npm", "install", "--save-dev"] + dependencies
    process = subprocess.run(install_cmd, capture_output=True, text=True)

    if process.returncode != 0:
        log.info(f"Error installing dependencies: {process.stderr}")
        return False

    # Create hardhat.config.js with relative paths for sources and artifacts
    hardhat_config = '''
module.exports = {
  solidity: {
    version: "0.8.20",
    settings: {
      optimizer: {
        enabled: true,
        runs: 200
      },
      evmVersion: "paris"
    }
  },
  paths: {
    sources: "./contracts",
    artifacts: "./artifacts"
  },
  networks: {
    hardhat: {
      chainId: 1337
    }
  },
  mocha: {
    timeout: 40000
  }
};
'''

    with open('hardhat.config.js', 'w') as f:
        f.write(hardhat_config)

    log.info("Hardhat environment setup complete")
    return True

setup_hardhat_environment()
def evaluate_contract_hard(contract_source, full_response):
    """
    Evaluate a Solidity contract by resolving additional dependencies,
    compiling it with Hardhat, and validating the generated artifact.

    Returns:
        dict: {
            "reward": int (10 for success, negative for errors),
            "error_token_indices": list of token indices corresponding to problematic error snippets
        }
    """
    try:
        # Write the contract source to a file in the contracts directory
        contract_path = Path('./contracts/MainContract.sol')
        with open(contract_path, 'w') as f:
            f.write(contract_source)

        # Detect contract name
        contract_name_match = re.search(r'contract\s+(\w+)(?:\s+is\s+[^{]+)?\s*{', contract_source)
        if contract_name_match:
            contract_name = contract_name_match.group(1)
            log.info(f"Detected contract name: {contract_name}")
        else:
            contract_name = "MainContract"
            log.info(f"Could not detect contract name, using default: {contract_name}")

        # Detect import statements that may require additional npm packages
        import_pattern = re.compile(r'import\s+(?:{[^}]*}\s+from\s+)?[\'"](.+?)[\'"];')
        imports = import_pattern.findall(contract_source)

        # Map import prefixes to their corresponding npm package names
        dependency_map = {
            '@openzeppelin/': '@openzeppelin/contracts',
            '@chainlink/': '@chainlink/contracts',
            '@uniswap/': '@uniswap/v3-core',
            'hardhat/': '@nomicfoundation/hardhat-toolbox',
            '@aave/': '@aave/protocol-v2',
            '@compound-finance/': '@compound-finance/contracts',
            '@balancer-labs/': '@balancer-labs/v2-solidity-utils',
            '@1inch/': '@1inch/solidity-utils',
            '@sushiswap/': '@sushiswap/core'
        }

        dependencies_to_install = set()
        for imp in imports:
            for prefix, package in dependency_map.items():
                if prefix in imp:
                    dependencies_to_install.add(package)

        if dependencies_to_install:
            log.info(f"Installing additional dependencies: {', '.join(dependencies_to_install)}")
            install_cmd = ['npm', 'install', '--save-dev'] + list(dependencies_to_install)
            install_process = subprocess.run(install_cmd, capture_output=True, text=True)
            if install_process.returncode != 0:
                log.info(f"Warning: Failed to install some dependencies: {install_process.stderr}")

        # Clean the environment and compile the contract
        subprocess.run(['npx', 'hardhat', 'clean'], capture_output=True)
        compile_process = subprocess.run(
            ['npx', 'hardhat', 'compile', '--force'],
            capture_output=True,
            text=True
        )

        # If compilation fails, extract error token indices
        if compile_process.returncode != 0:
            log.info("Compilation failed")
            log.info(f"STDOUT: {compile_process.stdout}")
            log.info(f"STDERR: {compile_process.stderr}")

            # Extract error snippets using regex matching lines starting with a number then "|"
            stderr = compile_process.stderr
            pattern = r'^\s*\d+\s*\|\s*(.+)$'
            error_snippets = re.findall(pattern, stderr, re.MULTILINE)
            log.info(f"Extracted error snippets: {error_snippets}")
            error_token_indices = []
            # Use offset mapping for precise mapping.
            encoded = tokenizer(full_response, return_offsets_mapping=True)
            offsets = encoded.get('offset_mapping', None)
            if offsets and isinstance(offsets[0], int):
                # This is a single list of integers; wrap it.
                offsets = [tuple(offsets)]
            elif offsets and isinstance(offsets[0], (list, tuple)) and isinstance(offsets[0][0], int):
                # Offsets are already in the correct format; do nothing.
                pass
            else:
              log.info("Offset mapping not available or in unexpected format: %s. Falling back to cumulative-length mapping.", repr(offsets))
              # Fallback: create offsets by cumulative token lengths.
              tokens = tokenizer(full_response, return_tensors="pt").input_ids[0]
              decoded_tokens = tokenizer.batch_decode(tokens, skip_special_tokens=True)
              offsets = []
              current_pos = 0
              for token in decoded_tokens:
                  token_len = len(token)
                  offsets.append((current_pos, current_pos + token_len))
                  current_pos += token_len
            # For each error snippet, find its character span and then map to token indices.
            for snippet in error_snippets:
                snippet_start = full_response.find(snippet)
                if snippet_start != -1:
                    snippet_end = snippet_start + len(snippet)
                    for i, (token_start, token_end) in enumerate(offsets):
                        if token_end > snippet_start and token_start < snippet_end:
                            error_token_indices.append(i)

            error_token_indices = sorted(set(error_token_indices))
            return {"reward": -1, "error_token_indices": error_token_indices}

        # Verify that the artifact was created
        artifact_dir = Path('artifacts/contracts/MainContract.sol')
        artifact_files = list(artifact_dir.glob('*.json'))
        if not artifact_files:
            log.info("Compilation appeared successful but no artifacts were generated")
            return {"reward": 5, "error_token_indices": []}

        log.info("Contract compiled successfully")
        return {"reward": 0.01, "error_token_indices": []}

    except Exception as e:
        log.info(f"Error during contract evaluation: {str(e)}")
        return {"reward": -20, "error_token_indices": []}

    finally:
        import shutil
        contract_path = Path('contracts/MainContract.sol')
        if contract_path.exists():
            os.remove(contract_path)
        artifacts_path = Path('artifacts/contracts/MainContract.sol')
        if artifacts_path.exists():
            shutil.rmtree(artifacts_path)



def evaluate_contract(contract_source):
    return evaluate_contract_hard(contract_source)

In [10]:
model.generation_config = GenerationConfig.from_model_config(model.config)

# Instantiate our simple reward model using the model's hidden size.
reward_model = SimpleRewardModel(hidden_size=model.config.hidden_size)

train_dataset = HFDataset.from_dict({"query": prompts})

# Define a prompt processor class (required by PPOTrainer)
class PromptProcessor:
    def __init__(self, processing_class):
        self.processing_class = processing_class

    def process_prompt(self, prompt):
        """Format prompt according to official Qwen pattern"""
        messages = [
            {"role": "system", "content": "You are an expert Solidity developer specialized in writing secure, efficient smart contracts. These contracts are intended to test your capabilities so ensure that you always provide complete, working code with no placeholders."},
            {"role": "user", "content": prompt}
        ]

        # Apply chat template exactly as in quickstart
        text = self.processing_class.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Create inputs as shown in quickstart
        model_inputs = self.processing_class([text], return_tensors="pt").to("cuda:0")
        return model_inputs

    def generate_response(self, prompt_tensors, **generation_kwargs):
        # Define generation parameters
        gen_kwargs = {
            "max_new_tokens": 2000,
            "temperature": 0.7,
            "top_p": 0.95,
            "do_sample": True,
            "repetition_penalty": 1.1
        }
        gen_kwargs.update(generation_kwargs)

        # Generate response
        with torch.amp.autocast("cuda"):  # Mixed precision for efficiency
            response = model.generate(**prompt_tensors, **gen_kwargs)

        return response

# Initialize processor with your tokenizer
prompt_processor = PromptProcessor(tokenizer)

formatted_prompts = [{"prompt": prompt} for prompt in prompts]
train_dataset = HFDataset.from_list(formatted_prompts)

ppo_config = PPOConfig(
    learning_rate=1e-5,
    batch_size=1,
    mini_batch_size=1,
    num_ppo_epochs=4,
    gamma=1.0,
    lam=0.95,
    cliprange=0.2,
    cliprange_value=0.2,
    vf_coef=0.1,
    seed=42,
    output_dir=output_dir
)
class CustomPPOTrainer(PPOTrainer):
    def __init__(self, *args, **kwargs):
        # Initialize without calling parent init as before
        from transformers import TrainingArguments
        self.args = kwargs.pop('args', None) or TrainingArguments("./output")
        self.policy_model = kwargs.pop('model')
        self.ref_model = kwargs.pop('ref_model')
        self.reward_function = kwargs.pop('reward_model')
        self.train_dataset = kwargs.pop('train_dataset')
        self.processing_class = kwargs.pop('processing_class')
        self.policy_model.gradient_checkpointing_enable()
        self.setup_data_loader()
        # Initialize the GradScaler for mixed precision training
        self.scaler = GradScaler()
        self.setup_optimizers()  # Modified to use a memory-efficient optimizer

    def setup_data_loader(self):
        from torch.utils.data import DataLoader
        self.dataloader = DataLoader(
            self.train_dataset,
            batch_size=self.args.batch_size,
            shuffle=True
        )

    def setup_optimizers(self):
        # Instead of torch.optim.Adam, use the 8-bit Adam optimizer from bitsandbytes
        self.optimizer = Adam8bit(
            self.policy_model.parameters(),
            lr=self.args.learning_rate
        )

    def step(self, prompts, responses, rewards):
          # Ensure correct data type for responses and prompts
          if hasattr(responses, 'dtype') and responses.dtype == torch.float16:
              responses = responses.float()
          if hasattr(prompts, 'dtype') and prompts.dtype == torch.float16:
              prompts = prompts.float()

          # Turn off autocast during loss computation
          with torch.amp.autocast("cuda", enabled=False):
              loss = self.compute_loss(prompts, responses, rewards)

          # Backward pass
          loss.backward()

          # Convert gradients to correct dtype and step the optimizer
          for group in self.optimizer.param_groups:
              for p in group['params']:
                  if p.grad is not None:
                      p.grad = p.grad.to(p.dtype)

          self.optimizer.step()
          self.optimizer.zero_grad(set_to_none=True)

          return {"loss": loss.item()}

    def compute_loss(self, prompt_ids, response_ids, rewards):
        pad_id = self.processing_class.pad_token_id
        attention_mask = (response_ids != pad_id).to(response_ids.device, dtype=torch.long)

        outputs = self.policy_model(
            input_ids=response_ids,
            attention_mask=attention_mask,
            use_cache=False
        )

        if isinstance(outputs, tuple):
            logits = outputs[0]
            value_predictions = outputs[1] if len(outputs) > 1 else None
        else:
            logits = outputs.logits
            value_predictions = outputs.value if hasattr(outputs, "value") else None

        new_policy_log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
        new_log_probs = new_policy_log_probs.gather(
            dim=-1, index=response_ids.unsqueeze(-1)
        ).squeeze(-1)

        with torch.no_grad():
            cpu_response_ids = response_ids.to("cpu")
            cpu_attention_mask = attention_mask.to("cpu")
            ref_outputs = self.ref_model(
                input_ids=cpu_response_ids,
                attention_mask=cpu_attention_mask,
                use_cache=False
            )

        if isinstance(ref_outputs, tuple):
            ref_logits = ref_outputs[0]
        else:
            ref_logits = ref_outputs.logits

        ref_logits = ref_logits.to(response_ids.device)
        old_policy_log_probs = torch.nn.functional.log_softmax(ref_logits, dim=-1)
        old_log_probs = old_policy_log_probs.gather(
            dim=-1, index=response_ids.unsqueeze(-1)
        ).squeeze(-1)

        ratios = torch.exp(new_log_probs - old_log_probs)

        rewards_tensor = torch.tensor(rewards, dtype=torch.float32, device=ratios.device)
        if rewards_tensor.dim() == 1:
            advantage = rewards_tensor.unsqueeze(1).expand_as(ratios)
        else:
            advantage = rewards_tensor.expand_as(ratios)

        clip_epsilon = 0.2
        clipped_ratios = torch.clamp(ratios, 1 - clip_epsilon, 1 + clip_epsilon)
        surrogate_loss = torch.min(ratios * advantage, clipped_ratios * advantage)
        loss_policy = -torch.mean(surrogate_loss)

        if value_predictions is not None:
            loss_value = torch.nn.functional.mse_loss(value_predictions.squeeze(-1), rewards_tensor)
        else:
            loss_value = 0.0

        kl_div = torch.nn.functional.kl_div(
            new_policy_log_probs, old_policy_log_probs.exp(), reduction='batchmean'
        )
        kl_coef = 0.1
        loss = loss_policy + loss_value + kl_coef * kl_div

        # Debug logging to show intermediate loss values
        log.info(f"compute_loss: loss_policy={loss_policy.item():.4f}, loss_value={loss_value:.4f}, kl_div={kl_div.item():.4f}, total_loss={loss.item():.4f}")
        return loss



# Use the custom trainer if needed
ppo_trainer = CustomPPOTrainer(
    args=ppo_config,
    model=model,
    ref_model=reference_model,
    reward_model=evaluate_contract,
    train_dataset=train_dataset,
    processing_class=tokenizer
)

In [11]:
def generate_text(prompt, max_new_tokens=2000, temperature=0.7):
    """Generate text using the official Qwen pattern"""
    # Format messages properly
    messages = [
            {"role": "system", "content": "You are an expert Solidity developer specialized in writing secure, efficient smart contracts. "+
            "These contracts are intended to test your capabilities so ensure that you always provide complete, working code that compiles with no placeholders. "
            "If there are values that you don't know, for example what to set an integer value to, just set it to any number. If you need a wallet id to accompilsh a task use GBH4TZYZ4IRCPO44CBOLFUHULU2WGALXTAVESQA6432MBJMABBB4GIYI"},
            {"role": "user", "content": prompt}
        ]
    # Apply chat template as shown in quickstart
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Create inputs
    model_inputs = tokenizer([text], return_tensors="pt").to("cuda:0")

    # Generate with proper settings
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            repetition_penalty=1.1
        )

    # Extract only the new tokens - this is key!
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode only the response portion
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [12]:
def extract_contract_code(response_text):
    """Extract the contract code from model response with improved pattern matching.
    Priority order:
    1. Find the last code block (```solidity ... ```)
    2. Look for Begin/End Smart Contract markers
    """
    # Case 1: Look for the LAST code block with optional 'solidity' language marker
    code_blocks = re.findall(r"```(?:solidity)?\s*(.*?)```", response_text, re.DOTALL)

    if code_blocks:
        # Get the last code block (highest priority)
        last_block = code_blocks[-1]
        if "contract" in last_block:
            start_marker = "// Begin Smart Contract"
            end_marker = "// End Smart Contract"
            return f"{start_marker}\n{last_block}\n{end_marker}"

    # Case 2: Look for markers outside or inside code blocks - use LAST occurrence
    start_marker = "// Begin Smart Contract"
    end_marker = "// End Smart Contract"

    # Find the LAST occurrence of the start marker
    start_idx = response_text.rfind(start_marker)

    # Find the LAST occurrence of the end marker
    end_idx = response_text.rfind(end_marker)

    if start_idx != -1 and end_idx != -1 and start_idx < end_idx:
        # Check if there's a closing bracket on the next line after the end marker
        end_marker_pos = end_idx + len(end_marker)
        next_closing_bracket = response_text[end_marker_pos:].find("}")

        if next_closing_bracket != -1 and response_text[end_marker_pos:end_marker_pos+next_closing_bracket].strip() == "":
            # Include the closing bracket if it's on the next line
            return response_text[start_idx:end_marker_pos + next_closing_bracket + 1]
        else:
            return response_text[start_idx:end_marker_pos]

    # Case 3: No markers and no code blocks
    log.info("No valid Solidity contract found in response")
    return None

In [13]:
# Function to clean up old checkpoints
def cleanup_old_checkpoints(checkpoint_dir="/content/drive/MyDrive/checkpoints",keep_last_n=2):
    # Get all checkpoint files sorted by creation time (newest first)
    all_checkpoints = sorted(
        glob.glob(f"{checkpoint_dir}/checkpoint_*.pt"),
        key=os.path.getctime,
        reverse=True
    )

    # Get all model directories sorted by creation time (newest first)
    all_model_dirs = sorted(
        [d for d in glob.glob(f"{checkpoint_dir}/model_epoch_*_prompt_*") if os.path.isdir(d)],
        key=os.path.getctime,
        reverse=True
    )

    # Keep the N newest checkpoints, delete the rest
    if len(all_checkpoints) > keep_last_n:
        for old_checkpoint in all_checkpoints[keep_last_n:]:
            try:
                os.remove(old_checkpoint)
                log.info(f"Deleted old checkpoint: {old_checkpoint}")
            except Exception as e:
                log.warning(f"Failed to delete {old_checkpoint}: {e}")

    # Keep the N newest model directories, delete the rest
    if len(all_model_dirs) > keep_last_n:
        for old_model_dir in all_model_dirs[keep_last_n:]:
            try:
                shutil.rmtree(old_model_dir)
                log.info(f"Deleted old model directory: {old_model_dir}")
            except Exception as e:
                log.warning(f"Failed to delete {old_model_dir}: {e}")


In [14]:
def apply_token_level_rewards(contract_code, error_lines, overall_reward, tokenizer):
    """
    Tokenize the extracted Solidity contract (which exactly matches the compiled file)
    and assign a reward to each token based on whether its line number is in error_lines.
    """
    # Tokenize the contract code (only the contract, not any extra prompt text)
    contract_tokens = tokenizer(contract_code, return_tensors="pt").input_ids[0]
    token_rewards = []
    current_line_num = 1

    # Process each token, updating the current line count based on newline characters
    for token_id in contract_tokens:
        token_text = tokenizer.decode([token_id])
        # If this token belongs to an error line, assign a negative reward
        if current_line_num in error_lines:
            token_rewards.append(-10)
        else:
            # Use a small positive reward if overall reward is positive; otherwise neutral (0)
            token_rewards.append(0 if overall_reward < 0 else 1)
        # Increase the current line count by the number of newline characters in the token
        current_line_num += token_text.count('\n')

    # Sanity check: ensure token_rewards length matches the number of tokens
    assert len(token_rewards) == len(contract_tokens), "Mismatch in tokens and reward mapping!"
    return contract_tokens.unsqueeze(0), token_rewards

In [15]:
def validate_prompt(prompt, model, tokenizer):
    """Run the model on a validation prompt and return the reward and error flag (1 if error, 0 if not)."""
    full_response = generate_text(prompt, max_new_tokens=2000, temperature=0.7)
    log.info("Validation full response:\n" + full_response)
    contract_code = extract_contract_code(full_response)
    if not contract_code:
        log.info("No contract code extracted for validation prompt")
        return -10, 1
    eval_result = evaluate_contract_hard(contract_code, full_response)
    overall_reward = eval_result["reward"]
    error_flag = 1 if overall_reward < 0 else 0
    log.info(f"Validation evaluation result: reward = {overall_reward}")
    return overall_reward, error_flag

In [16]:
def find_latest_checkpoint(checkpoint_dir):
    """Find and load the latest checkpoint if available."""
    checkpoint_files = glob.glob(f"{checkpoint_dir}/checkpoint_*.pt")
    if not checkpoint_files:
        return None, 0, 0, [], 0

    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    match = re.search(r'checkpoint_epoch_(\d+)_prompt_(\d+)', latest_checkpoint)

    if not match:
        return None, 0, 0, [], 0

    starting_epoch = int(match.group(1))
    starting_prompt_idx = int(match.group(2)) + 1  # Start from the next prompt

    # Load checkpoint
    checkpoint = torch.load(latest_checkpoint)
    compilation_errors = checkpoint.get('compilation_errors', [])
    global_step = checkpoint.get('global_step', 0)

    print(f"Resuming from checkpoint: {latest_checkpoint}")
    print(f"Starting from epoch {starting_epoch+1}, prompt {starting_prompt_idx+1}")

    return latest_checkpoint, starting_epoch, starting_prompt_idx, compilation_errors, global_step

def load_checkpoint(checkpoint_path, model, ppo_trainer):
    """Load model and optimizer state from checkpoint."""
    if not checkpoint_path:
        return

    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    ppo_trainer.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


def process_prompt(prompt, model, tokenizer, ppo_trainer, epoch, prompt_idx, compilation_errors):
    # Generate full response from the model.
    full_response = generate_text(prompt, max_new_tokens=2000, temperature=0.7)
    log.info("Full response:\n" + full_response)

    # Extract the contract code from the full response.
    contract_code = extract_contract_code(full_response)
    if not contract_code:
        log.info("No contract code extracted, assigning negative reward")
        if epoch < len(compilation_errors):
            compilation_errors[epoch] += 1
        return -10
    log.info("Extracted contract code:\n" + contract_code)

    # Evaluate the contract by compiling it, installing dependencies, etc.
    # This function now returns a dictionary with keys: "reward" and "error_token_indices".
    eval_result = evaluate_contract_hard(contract_code, full_response)
    overall_reward = eval_result["reward"]
    error_token_indices = eval_result["error_token_indices"]
    log.info(f"Evaluation result: reward = {overall_reward}, error token indices = {error_token_indices}")

    # Prepare the prompt inputs.
    prompt_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

    # Tokenize the extracted contract code exactly as compiled.
    contract_tokens = tokenizer(contract_code, return_tensors="pt").input_ids[0]
    # Build token-level rewards based on error token indices.
    token_rewards = []
    for i, token_id in enumerate(contract_tokens):
        if i in error_token_indices:
            token_rewards.append(-1)
        else:
            # If overall_reward is negative, assign 0; if positive, assign a small positive reward.
            token_rewards.append(0 if overall_reward < 0 else 0.01)
    assert len(token_rewards) == contract_tokens.shape[0], "Mismatch between token count and reward vector length"

    # Update the model using a sliding-window approach over the tokenized contract.
    update_model_with_sliding_window(prompt_inputs, contract_tokens.unsqueeze(0).to("cuda:0"), token_rewards, ppo_trainer)
    torch.cuda.empty_cache()

    return overall_reward



def update_model_with_sliding_window(inputs, contract_tokens, token_rewards, ppo_trainer):
    """
    Update the model using a sliding window approach, computing the average token-level reward
    for each window and then updating the model via PPO. Extra logging provides visibility into
    how the error rewards affect the loss.
    """
    window_size = 256
    step_size = 128  # 50% overlap

    for start_idx in range(0, contract_tokens.size(1), step_size):
        end_idx = min(start_idx + window_size, contract_tokens.size(1))
        if end_idx - start_idx < 32:
            continue  # Skip very short windows

        window_ids = contract_tokens[:, start_idx:end_idx].to("cuda:0")
        window_token_rewards = token_rewards[start_idx:end_idx]
        avg_window_reward = sum(window_token_rewards) / len(window_token_rewards)

        stats = ppo_trainer.step(
            inputs.input_ids,
            window_ids,
            [avg_window_reward]
        )
        log.info(f"Window {start_idx}-{end_idx}: avg_token_reward = {avg_window_reward:.4f}, loss = {stats.get('loss', 0):.4f}")
        torch.cuda.empty_cache()


def update_model_with_sliding_window(inputs, contract_tokens, token_rewards, ppo_trainer):
    """
    Update the model using a sliding window approach where the reward for each window
    is computed as the average of the token-level rewards. Extra logging is added
    so you can see the average reward and the resulting loss for each window.
    """
    window_size = 256
    step_size = 128  # 50% overlap between windows

    for start_idx in range(0, contract_tokens.size(1), step_size):
        end_idx = min(start_idx + window_size, contract_tokens.size(1))
        if end_idx - start_idx < 32:  # Skip very small chunks
            continue

        window_ids = contract_tokens[:, start_idx:end_idx].to("cuda:0")
        # Compute the average reward over tokens in this window
        window_token_rewards = token_rewards[start_idx:end_idx]
        avg_window_reward = sum(window_token_rewards) / len(window_token_rewards)

        stats = ppo_trainer.step(
            inputs.input_ids,
            window_ids,
            [avg_window_reward]
        )

        # Log additional details for visibility
        log.info(f"Window {start_idx}-{end_idx}: avg_token_reward: {avg_window_reward:.4f}, loss: {stats.get('loss', 0):.4f}")
        torch.cuda.empty_cache()


def save_checkpoint(checkpoint_dir, epoch, prompt_idx, global_step, model, ppo_trainer, compilation_errors):
    """Save model checkpoint."""
    checkpoint_path = f"{checkpoint_dir}/checkpoint_epoch_{epoch}_prompt_{prompt_idx}.pt"
    torch.save({
        'global_step': global_step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': ppo_trainer.optimizer.state_dict(),
        'compilation_errors': compilation_errors,
    }, checkpoint_path)
    log.info(f"Saved checkpoint to {checkpoint_path}")

    # Optionally save in HF format as well (includes both base model and value head)
    model_save_path = f"{checkpoint_dir}/model_epoch_{epoch}_prompt_{prompt_idx}"
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)
    log.info(f"Saved model to {model_save_path}")

    # Clean up old checkpoints after successful save
    cleanup_old_checkpoints(checkpoint_dir=checkpoint_dir)


def save_final_model(checkpoint_dir, model, tokenizer, ppo_trainer, compilation_errors, global_step):
    """Save the final model and generate performance plot."""
    # Save final checkpoint
    final_checkpoint_path = f"{checkpoint_dir}/checkpoint_final.pt"
    torch.save({
        'global_step': global_step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': ppo_trainer.optimizer.state_dict(),
        'compilation_errors': compilation_errors,
    }, final_checkpoint_path)
    log.info(f"Saved final checkpoint to {final_checkpoint_path}")

    # Save final model in HF format
    model.save_pretrained(f"{checkpoint_dir}/model_final")
    tokenizer.save_pretrained(f"{checkpoint_dir}/model_final")



def plot_performance(num_epochs, training_errors, validation_errors):
    """Plot both training and validation compilation errors per epoch with data point labels."""
    epochs = range(1, num_epochs + 1)
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, training_errors, marker='o', linestyle='-', label='Training Compilation Errors')
    plt.plot(epochs, validation_errors, marker='o', linestyle='-', label='Validation Compilation Errors')
    # Annotate each data point with its value.
    for i, (train_err, val_err) in enumerate(zip(training_errors, validation_errors)):
        plt.text(i+1, train_err, str(train_err), ha='center', va='bottom')
        plt.text(i+1, val_err, str(val_err), ha='center', va='bottom')
    plt.title('Compilation Errors per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Number of Compilation Errors')
    plt.legend()
    plt.grid(True)
    plt.xticks(epochs)
    plt.savefig('/content/drive/MyDrive/compilation_errors.png')
    plt.show()

def train(prompts, val_prompts, num_epochs=5, save_every=20, checkpoint_dir="/content/drive/MyDrive/checkpoints"):
    """Main training function updated to use a validation set and log when a prompt that previously failed now compiles.
    Also tracks and returns training and validation compilation errors."""
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Tracking variables for each epoch.
    compilation_errors = [0 for _ in range(num_epochs)]
    val_errors = []  # To store the number of validation compilation errors per epoch
    global_step = 0

    # Track compile status for each prompt over epochs (None means not set).
    prev_compile_status = [None] * len(prompts)

    latest_checkpoint, starting_epoch, starting_prompt_idx, loaded_errors, loaded_step = find_latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        load_checkpoint(latest_checkpoint, model, ppo_trainer)
        compilation_errors = loaded_errors if loaded_errors else compilation_errors
        global_step = loaded_step

    try:
        for epoch in range(starting_epoch, num_epochs):
            log.info(f"Epoch {epoch+1}/{num_epochs}")
            current_compile_status = [None] * len(prompts)

            # Process training prompts.
            start_idx = starting_prompt_idx if epoch == starting_epoch else 0
            for i, prompt in enumerate(prompts[start_idx:], start=start_idx):
                global_step += 1
                log.info(f"Processing prompt {i+1}/{len(prompts)}")
                overall_reward = process_prompt(prompt, model, tokenizer, ppo_trainer, epoch, i, compilation_errors)
                # Mark prompt as 'compiled' (True) if reward is positive.
                current_compile_status[i] = (overall_reward > 0)

                if global_step % save_every == 0:
                    save_checkpoint(checkpoint_dir, epoch, i, global_step, model, ppo_trainer, compilation_errors)

            print(f"Number of training compilation errors for this epoch: {compilation_errors[epoch]}")

            # Log if any prompt that failed in the previous epoch now compiles.
            if prev_compile_status[0] is not None:
                for i, (prev_status, curr_status) in enumerate(zip(prev_compile_status, current_compile_status)):
                    if prev_status is False and curr_status is True:
                        log.info(f"Prompt {i+1} now compiles but previously did not.")
            prev_compile_status = current_compile_status

            # Evaluate on the validation set.
            epoch_val_errors = 0
            if val_prompts:
                for val_prompt in val_prompts:
                    _, error_flag = validate_prompt(val_prompt, model, tokenizer)
                    epoch_val_errors += error_flag
                log.info(f"Epoch {epoch+1} validation compilation errors: {epoch_val_errors}")
            else:
                epoch_val_errors = 0
            val_errors.append(epoch_val_errors)
    finally:
        save_final_model(checkpoint_dir, model, tokenizer, ppo_trainer, compilation_errors, global_step)
        plot_performance(num_epochs, compilation_errors, val_errors)
    return compilation_errors, val_errors


In [ ]:
try:
  train(prompts,validation_prompts)
finally:
  import shutil
  shutil.copy("/content/logger.log","/content/drive/MyDrive/log.log")